# Calcula o fator R para dados de estações com intervalo de 5 minutos <br>
por [Marina Galdez de Castro Silva](mailto:marinagaldez@id.uff.br) <br>
2023
<br>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
# Carregar os dados do Excel no Google Colab
filename = 'Ponto2.xlsx'
df = pd.read_excel(filename)

# Dados organizados em excel com colunas: ANO	MÊS	DIA	HORA	DATA_HORA	PRECIPITACAO
# Exemplo: 2018	12	18	11:39:04	18/12/18 11:39	2,2

In [ ]:
print(df.shape)

(192704, 6)


In [ ]:
df

,ANO,MÊS,DIA,HORA,DATA_HORA,PRECIPITACAO
0,2018,12,18,11:39:04,18/12/18 11:39:04,2.2
1,2018,12,18,11:44:04,18/12/18 11:44:04,0
2,2018,12,18,11:49:04,18/12/18 11:49:04,0
3,2018,12,18,11:54:04,18/12/18 11:54:04,0
4,2018,12,18,11:59:04,18/12/18 11:59:04,0
...,...,...,...,...,...,...
192699,2020,11,23,14:15:00,23/11/20 14:15:00,0
192700,2020,11,23,14:20:00,23/11/20 14:20:00,0
192701,2020,11,23,14:25:00,23/11/20 14:25:00,0
192702,2020,11,23,14:30:00,23/11/20 14:30:00,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192704 entries, 0 to 192703
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ANO           192704 non-null  int64 
 1   MÊS           192704 non-null  int64 
 2   DIA           192704 non-null  int64 
 3   HORA          192704 non-null  object
 4   DATA_HORA     192704 non-null  object
 5   PRECIPITACAO  192704 non-null  object
dtypes: int64(3), object(3)
memory usage: 8.8+ MB


In [ ]:
# Convertendo a coluna 'PRECIPITACAO' para float
df['PRECIPITACAO'] = pd.to_numeric(df['PRECIPITACAO'].replace(',', '.', regex=True), errors='coerce')

# Convertendo a coluna 'HORA' para datetime
df['HORA'] = pd.to_datetime(df['HORA'], format='%H:%M:%S').dt.time

# Convertendo a coluna 'DATA_HORA' para datetime
df['DATA_HORA'] = pd.to_datetime(df['DATA_HORA'], format='%d/%m/%y %H:%M:%S')

# Verificando os tipos de dados após as conversões
print(df.dtypes)


ANO                      int64
MÊS                      int64
DIA                      int64
HORA                    object
DATA_HORA       datetime64[ns]
PRECIPITACAO           float64
dtype: object


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192704 entries, 0 to 192703
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   ANO           192704 non-null  int64         
 1   MÊS           192704 non-null  int64         
 2   DIA           192704 non-null  int64         
 3   HORA          192704 non-null  object        
 4   DATA_HORA     192704 non-null  datetime64[ns]
 5   PRECIPITACAO  192704 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 8.8+ MB


In [ ]:
df.head()

,ANO,MÊS,DIA,HORA,DATA_HORA,PRECIPITACAO
0,2018,12,18,11:39:04,2018-12-18 11:39:04,2.2
1,2018,12,18,11:44:04,2018-12-18 11:44:04,0.0
2,2018,12,18,11:49:04,2018-12-18 11:49:04,0.0
3,2018,12,18,11:54:04,2018-12-18 11:54:04,0.0
4,2018,12,18,11:59:04,2018-12-18 11:59:04,0.0


In [ ]:
# Criar uma nova coluna para identificar os eventos de chuva
df['Evento'] = 0

# Inicializar variáveis
evento_atual = 0
ultima_chuva = pd.to_datetime('1900-01-01 00:00:00')

# Iterar sobre as linhas dos dados
for index, row in df.iterrows():
    # Verificar se há precipitação
    if row['PRECIPITACAO'] > 0:
        # Verificar se é uma nova chuva
        if (row['DATA_HORA'] - ultima_chuva).total_seconds() / 3600 > 6:
            evento_atual += 1

        # Atualizar a coluna 'Evento'
        df.at[index, 'Evento'] = evento_atual

        # Atualizar a última chuva
        ultima_chuva = row['DATA_HORA']

# Salvar o DataFrame de volta no Excel
df.to_excel('chuva_com_eventos.xlsx', index=False)

Se o intervalo de dados de precipitacao for diferente de 5 minutos, mexer na celula abaixo

In [ ]:
# Carregar os dados do Excel
df2 = pd.read_excel('chuva_com_eventos.xlsx')

# Criar uma nova coluna para identificar chuvas individuais erosivas
df2['erosivos'] = 0

# Inicializar o número do evento erosivo
num_erosivo = 1

# Iterar sobre os eventos para identificar os erosivos
for evento in df2['Evento'].unique():
    df_evento = df2[df2['Evento'] == evento]

    # Calcular a soma da precipitação total
    precipitacao_total = df_evento['PRECIPITACAO'].sum()

    # Verificar se há 15 minutos com chuva >= 6.35 mm
    chuva_15min = df_evento['PRECIPITACAO'].rolling(window=3).sum() >= 6.35  # Considerando que cada linha representa 5 minutos

    # Verificar as condições
    if precipitacao_total >= 12.7 or chuva_15min.any():
        df2.loc[df2['Evento'] == evento, 'erosivos'] = num_erosivo
        num_erosivo += 1

# Salvar o DataFrame de volta no Excel
df2.to_excel('chuva_com_eventos_erosivos.xlsx', index=False)


Se o intervalo for diferente de 5 minutos, mexer na celula abaixo também

In [ ]:
# Carregar os dados do Excel
df3 = pd.read_excel('chuva_com_eventos_erosivos.xlsx')

# Adicionar coluna 'Soma_30min' para armazenar a soma de precipitação em intervalos de 30 minutos
df3['Soma_30min'] = 0

# Inicializar variáveis para o cálculo da soma
tempo_atual = pd.Timedelta(0)
soma_tempo = 0
last_precip_index = None

# Intervalo de tempo desejado (em minutos)
intervalo_tempo = pd.Timedelta(minutes=30)

# Iterar sobre todas as linhas do DataFrame
for index, row in df3.iterrows():
    # Verificar se é um evento "erosivo"
    if row['erosivos'] != 0:
        # Calcular a diferença de tempo entre a linha atual e a última com precipitação
        diff_tempo = (row['DATA_HORA'] - df3.loc[last_precip_index, 'DATA_HORA']).total_seconds() / 60 if last_precip_index is not None else 0

        # Verificar se passou o intervalo desejado ou é a última linha do evento
        if (tempo_atual + pd.Timedelta(minutes=diff_tempo)) >= intervalo_tempo or index == df3.index[-1]:
            # Adicionar a soma da precipitação ao DataFrame original
            if last_precip_index is not None:
                df3.loc[last_precip_index, 'Soma_30min'] = soma_tempo

            # Reiniciar variáveis para o próximo intervalo de 30 minutos
            tempo_atual = pd.Timedelta(0)
            soma_tempo = 0

        # Atualizar o índice da última linha com precipitação
        last_precip_index = index

    # Atualizar a soma apenas se o valor de precipitação for válido e dentro do intervalo desejado
    if not pd.isna(row['PRECIPITACAO']) and tempo_atual < intervalo_tempo:
        soma_tempo += row['PRECIPITACAO']
        tempo_atual += pd.Timedelta(minutes=5)  # Considerando que cada linha representa 5 minutos

# Adicionar a soma da precipitação ao DataFrame para o último evento erosivo
if last_precip_index is not None:
    df3.loc[last_precip_index, 'Soma_30min'] = soma_tempo

# Salvar o DataFrame de volta no Excel
df3.to_excel('chuva_com_eventos_erosivos_soma.xlsx', index=False)

In [ ]:
# Carregar os dados do Excel
df4 = pd.read_excel('chuva_com_eventos_erosivos_soma.xlsx')

# Adicionar coluna 'I' (Intensidade) multiplicando a coluna 'Soma_30min' por 2
df4['I'] = df4['Soma_30min'] * 2

# Adicionar coluna 'E' (Energia cinética) utilizando a fórmula, considerando apenas se 'I' é maior que 0
df4['E'] = np.where(df4['I'] > 0, 0.119 + 0.0873 * np.log(df4['I']), 0)

# Adicionar coluna 'E*P' utilizando a fórmula, considerando apenas se 'I' é maior que 0
df4['E*P'] = df4['E']*df4['Soma_30min']

# Salvar o DataFrame de volta no Excel
df4.to_excel('chuva_com_eventos_erosivos_I_E.xlsx', index=False)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Carregar os dados do Excel
df5 = pd.read_excel('chuva_com_eventos_erosivos_I_E.xlsx')

# Calcular a soma de 'E*P' para cada evento erosivo e armazenar apenas na última linha
df5['EcT'] = df5.groupby('erosivos')['E*P'].transform('sum')

# Identificar a última linha de cada evento erosivo
ultima_linha_evento = df5.groupby('erosivos').tail(1).index

# Ajustar a coluna 'EcT' para conter o valor apenas na última linha de cada evento erosivo
df5['EcT'] = np.where(df5.index.isin(ultima_linha_evento), df5['EcT'], 0)

# Salvar o DataFrame de volta no Excel
df5.to_excel('chuva_com_eventos_erosivos_I_E_EcT.xlsx', index=False)

In [ ]:
# Carregar os dados do Excel
df6 = pd.read_excel('chuva_com_eventos_erosivos_I_E_EcT.xlsx')

# Calcular o máximo de 'I' para cada evento erosivo
max_I_por_evento = df6.groupby('erosivos')['I'].transform('max')

# Multiplicar 'EcT' pelo máximo de 'I' e armazenar em 'EI30'
df6['EI30'] = df6['EcT'] * max_I_por_evento

# Salvar o DataFrame de volta no Excel
df6.to_excel('chuva_com_eventos_erosivos_I_E_EcT_EI30.xlsx', index=False)

In [ ]:
# Carregar os dados do Excel
df7 = pd.read_excel('chuva_com_eventos_erosivos_I_E_EcT_EI30.xlsx')

# Extrair o mês e ano de 'DATA_HORA'
df7['MES_ANO'] = df7['DATA_HORA'].dt.to_period('M')

# Calcular a soma de 'EI30' para cada mês/ano
EI30_por_mes_ano = df7.groupby('MES_ANO')['EI30'].sum().reset_index()

# Salvar a tabela separada no Excel
EI30_por_mes_ano.to_excel('EI30_por_mes_ano.xlsx', index=False)

In [ ]:
EI30_por_mes_ano.head()

,MES_ANO,EI30
0,2018-12,36.196982
1,2019-01,382.075119
2,2019-02,60.295800
3,2019-03,0.000000
4,2019-04,0.000000


In [ ]:
EI30_por_mes_ano.describe()

,EI30
count,24.000000
mean,434.144747
std,585.317304
min,0.000000
25%,54.793941
50%,217.165110
75%,528.741078
max,2267.308445


In [ ]:
# Extrair o mês de 'MES_ANO'
EI30_por_mes_ano['MES'] = EI30_por_mes_ano['MES_ANO'].dt.month

# Calcular a média mensal considerando todos os anos
media_mensal_geral = EI30_por_mes_ano.groupby('MES')['EI30'].mean().reset_index()

In [ ]:
media_mensal_geral

,MES,EI30
0,1,562.429401
1,2,326.711198
2,3,0.000000
3,4,19.144183
4,5,385.740993
5,6,131.968919
6,7,786.197022
7,8,126.341383
8,9,214.982102
9,10,1033.089582


In [ ]:
# Extrair o ano de 'MES_ANO'
EI30_por_mes_ano['ANO'] = EI30_por_mes_ano['MES_ANO'].dt.year

# Calcular a soma total do ano para cada ano
soma_ano = EI30_por_mes_ano.groupby('ANO')['EI30'].sum().reset_index()

In [ ]:
soma_ano

,ANO,EI30
0,2018,36.196982
1,2019,5651.757938
2,2020,4731.519001


In [ ]:
# Valor de R

# Calcular a soma total da coluna 'EI30'
soma_total = EI30_por_mes_ano['EI30'].sum()

# Calcular o número único de anos na coluna 'ANO'
#num_anos_unicos = EI30_por_mes_ano['ANO'].nunique()
num_anos_unicos = 2 # porque começou em dezembro de 2018 e foi até novembro de 2020

# Calcular R
R = soma_total / num_anos_unicos

# Imprimir o valor da soma total
print("Soma Total de EI30: ", soma_total)

# Imprimir o valor da R
print("R em MJ mm ha-1 h-1 ", R)

Soma Total de EI30:  10419.473921213375
R em MJ mm ha-1 h-1  5209.736960606688


In [ ]:
R

5209.736960606688